In [1]:
import pdb
import cv2
import numpy as np

In [2]:
# Function to visualize the flow
def draw_flow(img, flow, step=16):
    
    # get the height and width of the image
    h, w = img.shape[:2]
    
    # get the y, x pixel values and stores them as a grid
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)

    # get the displacement(horizontal and vertical) for each pixel with corresponding x and y values
    fx, fy = flow[y,x].T    
    
    #get a 2x2 matrix for every x, y value with the initial value and value with the displacement
    lines = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2)
    
    # round the new x, y values to the next highest value and convert them to int
    lines = np.int32(lines + 0.5)
    
    # convert the input image to gray
    vis = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    
    #draw multiple lines(indicators) on the input image
    cv2.polylines(vis, lines, 0, (0, 255, 0))
    
    # draw a circle for all the points with radius 1, note lines is a 2x2 matrix for every pixel
    for (x1, y1), (x2, y2) in lines:
        cv2.circle(vis, (x1, y1), 1, (0, 255, 0), -1)
    return vis


#### The following code compares the current and the previous frames at every instance of time

In [6]:
# Create object for video capture
cap = cv2.VideoCapture("my_vids/VID-20180512-WA0006.mp4")

# Get the image frame and a return value[true/false]
ret, frame = cap.read()

# Initialize a counter to keep track of the frames and decide when to change the initial frame
frame_count = 0

# Transpose the frame to view in potrait mode
frame = cv2.transpose(frame)

# Convert the captured frame to gray and store it as previous frame
gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

# Enter the while condition only if we have a previous frame to compare against
while(1):
    # change the old frame after every 15 or 35 frames or if it is the initial frame
    if(frame_count == 0 or frame_count%35 == 0):
        prev_gray = gray
    
# Obtain the new frame and return value[true/false]
    ret, frame = cap.read()
    
    if ret:
        
        # increment the frame counter if the return value is true
        frame_count+= 1
    
        # Transpose this frame to ensure proper comparison between the previous and new frames
        frame = cv2.transpose(frame)
    
        # Convert the new frame to gray and store it in 
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
        # Calculate the optical flow between the two frames using farneback method
        flow = cv2.calcOpticalFlowFarneback(prev_gray,gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        
        #get the magnitude and angle of flow vectors in polar form, i.e the angle is in radians
        mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
        
        #normalize the magnitude between the minimum and maximum values.
        mag = cv2.normalize(mag,None,cv2.NORM_MINMAX)
              
        # Visualize the optical flow
        cv2.imshow('flow',draw_flow(gray, flow))
    
        # Waits for 30 milliseconds for a keypress
        ch = cv2.waitKey(30) & 0xff
    
        # Break if escape key is used 
        if ch == 27:
            break
        
        # Do a frame capture and store as image if 's' is pressed
        elif ch == ord('s'):
            cv2.imwrite('opticalfb.png',new_frame)
            
    # stop if there are no more frames to calculate optical flow        
    else :
        break

    
# Release the capture mode
cap.release()

# Close all windows and exit
cv2.destroyAllWindows()

In [45]:
cap = cv2.VideoCapture("VID-20180512-WA0006.mp4")
cap.set(cv2.CAP_PROP_FPS, 60)
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps    = cap.get(cv2.CAP_PROP_FPS)
print "number of frames = ", length, ',',"FPS =", fps 
cap.release()
cv2.destroyAllWindows()

number of frames =  561 , FPS = 30.147


Why cluster the optical flow?


If we are clustering shouldn't it be before running the optical flow?


What to do after clustering?


The optical flow analysis i have shows almost no Flow components? 


should I reduce the FPS to get a better visualization?
